In [1]:
import warnings
warnings.filterwarnings("ignore")

import DataUtils.MATH as MATH
import DataUtils.MMLU as MMLU
from vLLMUtils import *
from hfUtils import *
from ModelUtils import *

import numpy as np

import torch

In [2]:
# tokenizer, model = load_model_and_tokenizer('meta-llama/Llama-3.1-8B-Instruct')

In [3]:
model = load_model('meta-llama/Llama-3.1-8B-Instruct')

WARNING 11-30 17:23:05 arg_utils.py:957] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 11-30 17:23:05 config.py:1021] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 11-30 17:23:05 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='meta-llama/Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=39808, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=au

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 11-30 17:23:16 model_runner.py:1067] Loading model weights took 14.9888 GB
INFO 11-30 17:23:16 gpu_executor.py:122] # GPU blocks: 2488, # CPU blocks: 2048
INFO 11-30 17:23:16 gpu_executor.py:126] Maximum concurrency for 39808 tokens per request: 1.00x
INFO 11-30 17:23:17 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-30 17:23:17 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-30 17:23:25 model_runner.py:1523] Graph capturing finished in 8 secs.


In [12]:
MMLU_inputs_chat, MMLU_Answers_chat = MMLU.get_test_data_chat_template()
# MATH_inputs_chat, MATH_Answers_chat = MATH.get_test_data_chat_template()

MMLU_inputs_plain, MMLU_Answers_plain = MMLU.get_test_data_plain()
# MATH_inputs_plain, MATH_Answers_plain = MATH.get_test_data_plain()

# This will take a few seconds
# tokenized_MMLU_inputs_chat = tokenize_chat(MMLU_inputs_chat, tokenizer)
# tokenized_MMLU_inputs_plain = tokenize_plain(MMLU_inputs_plain, tokenizer)

# tokenized_MATH_inputs_chat = tokenize_chat(MATH_inputs_chat, tokenizer)
# tokenized_MATH_inputs_plain = tokenize_plain(MATH_inputs_plain, tokenizer)

MMLU_inputs_chat = model.get_tokenizer().apply_chat_template(MMLU_inputs_chat, add_generation_prompt=True, tokenize=False)

In [1]:
MMLU_inputs_plain[0]

NameError: name 'MMLU_inputs_plain' is not defined

In [6]:
from vllm import SamplingParams
sp = SamplingParams(n=1, temperature=0.0, max_tokens=1, top_p=1.0, allowed_token_ids=[32,33,34,35])

In [7]:
results = model.generate(MMLU_inputs_chat, sampling_params=sp)

Processed prompts: 100%|██████████| 14042/14042 [18:23<00:00, 12.73it/s, est. speed input: 9606.80 toks/s, output: 12.73 toks/s]


In [10]:
answers = extract_outputs(results)

In [11]:
coorect = 0
for i, answer in enumerate(answers):
    if answer == MMLU_Answers_chat[i]:
        coorect += 1
print(f"Accuracy: {coorect/len(answers)}")

Accuracy: 0.5858852015382424


In [4]:
# print some sample tokenized inputs
rnd_idx = np.random.randint(0, len(MMLU_inputs_chat))
print("MMLU Chat Template Input: \n", tokenizer.decode(token_ids = tokenized_MMLU_inputs_chat[rnd_idx]))
print("MMLU Plain Input: \n", tokenizer.decode(token_ids = tokenized_MMLU_inputs_plain[rnd_idx]))

MMLU Chat Template Input: 
 <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You will be given multiple choice questions about high school government and politics.

 Here are some examples:

Uncertainty over the limits to presidential power is caused primarily by the fact that
A. the constitutional definition of those powers is broad and unspecific
B. most people agree that the Constitution places too many limits on presidential power
C. the Supreme Court consistently refuses to rule on cases concerning presidential powers
D. constitutional amendments have greatly increased presidential powers
Answer: \boxed{A}

The term "budget deficit" refers to the
A. annual increase in federal spending on the military
B. amount of interest on the national debt
C. difference between the initial budget proposals made by the president and Congress
D. amount the government spends in excess of its revenues
Answer: \boxed{D}

Whic

In [7]:
MMLU.get_input_kwargs(tokenizer)

{'force_words_ids': [[[59, 80175, 90, 32, 92],
   [59, 80175, 90, 33, 92],
   [59, 80175, 90, 34, 92],
   [59, 80175, 90, 35, 92]]],
 'max_new_tokens': 8,
 'num_beams': 4,
 'do_sample': False,
 'top_p': 1.0}

In [ ]:
results = run_model(tokenized_MMLU_inputs_chat, model, tokenizer, **MMLU.get_input_kwargs(tokenizer))

  0%|          | 0/14042 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask an